# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [24]:
import numpy as np
import os
from scipy.misc import imread, imresize
from skimage.transform import resize
# from imageio import imread
import datetime
import os
import warnings
warnings.filterwarnings('ignore')

from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout,Input
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers


We set the random seed so that the results don't vary drastically.

In [25]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.set_random_seed(30)
import keras

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [26]:
cd Project_data/

[Errno 2] No such file or directory: 'Project_data/'
/mnt/disks/user/project/PROJECT/Project_data


In [27]:
train_doc = np.random.permutation(open('train.csv').readlines())
val_doc = np.random.permutation(open('val.csv').readlines())
batch_size = 10 #experiment with the batch size

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [28]:
x = 30 # number of frames
y = 120 # image width
z = 120 # image height

In [29]:
x = 30 # number of frames
y = 120 # image width
z = 120 # image height

def generator(source_path, folder_list, batch_size):
#     print('3')
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [x for x in range(0,x)] #create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    
                    temp = resize(image,(120,120))
                    temp = temp/127.5-1 #Normalize data
                    
                    batch_data[folder,idx,:,:,0] = (temp[:,:,0]) #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = (temp[:,:,1]) #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = (temp[:,:,2]) #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

#             print('1')
        # write the code for the remaining data points which are left after full batches
        if (len(folder_list) != batch_size*num_batches):
            print("Batch: ",num_batches+1,"Index:", batch_size)
            batch_size = len(folder_list) - (batch_size*num_batches)
            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    temp = resize(image,(120,120))
                    temp = temp/127.5-1 #Normalize data
                    
                    batch_data[folder,idx,:,:,0] = (temp[:,:,0])
                    batch_data[folder,idx,:,:,1] = (temp[:,:,1])
                    batch_data[folder,idx,:,:,2] = (temp[:,:,2])
                   
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
#                 print('2')
            yield batch_data, batch_labels

Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [30]:
curr_dt_time = datetime.datetime.now()
train_path = 'train'
val_path = 'val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 10 # choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 10


## Model 1
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [31]:

#write your model here
model1 = Sequential()

model1.add(Conv3D(32, #number of filters 
                 kernel_size=(3,3,3), 
                 input_shape=(30, 120, 120, 3),
                 padding='same'))
model1.add(BatchNormalization())
model1.add(Activation('relu'))

model1.add(MaxPooling3D(pool_size=(2,2,2)))

model1.add(Conv3D(64, #Number of filters, 
                 kernel_size=(3,3,3), 
                 padding='same'))
model1.add(BatchNormalization())
model1.add(Activation('relu'))

model1.add(MaxPooling3D(pool_size=(2,2,2)))

model1.add(Conv3D(128, #Number of filters 
                 kernel_size=(1,3,3), 
                 padding='same'))
model1.add(BatchNormalization())
model1.add(Activation('relu'))

model1.add(MaxPooling3D(pool_size=(2,2,2)))

model1.add(Conv3D(256, #Number pf filters 
                 kernel_size=(1,3,3), 
                 padding='same'))
model1.add(BatchNormalization())
model1.add(Activation('relu'))

model1.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model1.add(Flatten())

model1.add(Dense(1000, activation='relu'))
model1.add(Dropout(0.25))

model1.add(Dense(500, activation='relu'))
model1.add(Dropout(0.25))

#softmax layer
model1.add(Dense(5, activation='softmax'))


Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [32]:
optimiser = optimizers.Adam(lr=0.001) #write your optimizer
model1.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model1.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_5 (Conv3D)            (None, 30, 120, 120, 32)  2624      
_________________________________________________________________
batch_normalization_5 (Batch (None, 30, 120, 120, 32)  128       
_________________________________________________________________
activation_5 (Activation)    (None, 30, 120, 120, 32)  0         
_________________________________________________________________
max_pooling3d_5 (MaxPooling3 (None, 15, 60, 60, 32)    0         
_________________________________________________________________
conv3d_6 (Conv3D)            (None, 15, 60, 60, 64)    55360     
_________________________________________________________________
batch_normalization_6 (Batch (None, 15, 60, 60, 64)    256       
_________________________________________________________________
activation_6 (Activation)    (None, 15, 60, 60, 64)    0         
__________

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [33]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [34]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, cooldown=1, verbose=1)# write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [35]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [36]:
# validation_steps

In [37]:
model1.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  val ; batch size = 10
Source path =  train ; batch size = 10
Epoch 1/10
67/67 [==============================] - 131s 2s/step - loss: 12.9739 - categorical_accuracy: 0.1821 - val_loss: 12.5721 - val_categorical_accuracy: 0.2200

Epoch 00001: saving model to model_init_2020-03-2615_56_14.668735/model-00001-12.94119-0.18401-12.57211-0.22000.h5
Epoch 2/10
67/67 [==============================] - 44s 650ms/step - loss: 12.9105 - categorical_accuracy: 0.1990 - val_loss: 12.5721 - val_categorical_accuracy: 0.2200

Epoch 00002: saving model to model_init_2020-03-2615_56_14.668735/model-00002-12.91051-0.19900-12.57211-0.22000.h5
Epoch 3/10
67/67 [==============================] - 44s 651ms/step - loss: 12.7501 - categorical_accuracy: 0.2090 - val_loss: 12.5721 - val_categorical_accuracy: 0.2200

Epoch 00003: saving model to model_init_2020-03-2615_56_14.668735/model-00003-12.75014-0.20896-12.57211-0.22000.h5

Epoch 00003: ReduceLROnPlateau reducing learning rate to 0.00050000002

In [38]:
classes = 5 #left swipe, right swipe, thumbs up, thumbs down, stop
channel = 3 #RGB
x = 30 # number of frames
y = 120 # image width
z = 120 # image height

def generator_ex(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [x for x in range(0,x)] #create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,x,y,z,channel)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,classes)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    
                    temp = resize(image,(y,z))
                    #Converting to gray scale
                    temp = temp.mean(axis=-1,keepdims=1) 
                    temp = temp/127.5-1 #Normalize data
                    batch_data[folder,idx] = temp #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
                
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        if (len(folder_list) != batch_size*num_batches):
            print("Batch: ",num_batches+1,"Index:", batch_size)
            batch_size = len(folder_list) - (batch_size*num_batches)
            batch_data = np.zeros((batch_size,x,y,z,channel)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,classes)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    temp = resize(image,(y,z))
                    #Converting to gray scale
                    temp = temp.mean(axis=-1,keepdims=1) 
                    temp = temp/127.5-1 #Normalize data
                    
                    batch_data[folder,idx] = temp
                   
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels


## Model 2

In [39]:
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam

# Define 2nd model 
model2 = Sequential()
model2.add(Conv3D(32, kernel_size=(3, 3, 3), input_shape=(x,y,z,channel), padding='same'))
model2.add(Activation('relu'))
model2.add(Conv3D(32, kernel_size=(3, 3, 3), padding='same'))
model2.add(Activation('relu'))
model2.add(MaxPooling3D(pool_size=(3, 3, 3), padding='same'))
model2.add(Dropout(0.25))

model2.add(Conv3D(64, kernel_size=(3, 3, 3), padding='same'))
model2.add(Activation('relu'))
model2.add(Conv3D(64, kernel_size=(3, 3, 3), padding='same'))
model2.add(Activation('relu'))
model2.add(MaxPooling3D(pool_size=(3, 3, 3), padding='same'))
model2.add(Dropout(0.25))

model2.add(Flatten())
model2.add(Dense(512, activation='relu'))
model2.add(Dropout(0.5))
model2.add(Dense(classes, activation='softmax'))

model2.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model2.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_9 (Conv3D)            (None, 30, 120, 120, 32)  2624      
_________________________________________________________________
activation_9 (Activation)    (None, 30, 120, 120, 32)  0         
_________________________________________________________________
conv3d_10 (Conv3D)           (None, 30, 120, 120, 32)  27680     
_________________________________________________________________
activation_10 (Activation)   (None, 30, 120, 120, 32)  0         
_________________________________________________________________
max_pooling3d_9 (MaxPooling3 (None, 10, 40, 40, 32)    0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 10, 40, 40, 32)    0         
_________________________________________________________________
conv3d_11 (Conv3D)           (None, 10, 40, 40, 64)    55360     
__________

In [40]:

train_generator = generator_ex(train_path, train_doc, batch_size)
val_generator = generator_ex(val_path, val_doc, batch_size)

In [41]:
model2.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path = Source path =  train ; batch size = Epoch 1/10
 10
val ; batch size = 10
67/67 [==============================] - 152s 2s/step - loss: 1.7025 - categorical_accuracy: 0.2159 - val_loss: 1.6089 - val_categorical_accuracy: 0.2100

Epoch 00001: saving model to model_init_2020-03-2615_56_14.668735/model-00001-1.70366-0.21116-1.60891-0.21000.h5
Epoch 2/10
67/67 [==============================] - 49s 739ms/step - loss: 1.6109 - categorical_accuracy: 0.1692 - val_loss: 1.6095 - val_categorical_accuracy: 0.1800

Epoch 00002: saving model to model_init_2020-03-2615_56_14.668735/model-00002-1.61092-0.16915-1.60948-0.18000.h5
Epoch 3/10
67/67 [==============================] - 50s 751ms/step - loss: 1.6126 - categorical_accuracy: 0.1891 - val_loss: 1.6068 - val_categorical_accuracy: 0.2300

Epoch 00003: saving model to model_init_2020-03-2615_56_14.668735/model-00003-1.61259-0.18905-1.60680-0.23000.h5
Epoch 4/10
67/67 [==============================] - 50s 747ms/step - loss: 1.6093 -

## <font color = 'green' >  trial 1 - Resizing changing x, y , z value

In [42]:
 
x = 30 # number of frames
y = 60 # image width
z = 60 # image height

In [43]:
# Define model b
model2 = Sequential()
model2.add(Conv3D(32, kernel_size=(3, 3, 3), input_shape=(x,y,z,channel), padding='same'))
model2.add(Activation('relu'))
model2.add(Conv3D(32, kernel_size=(3, 3, 3), padding='same'))
model2.add(Activation('relu'))
model2.add(MaxPooling3D(pool_size=(3, 3, 3), padding='same'))
model2.add(Dropout(0.25))

model2.add(Conv3D(64, kernel_size=(3, 3, 3), padding='same'))
model2.add(Activation('relu'))
model2.add(Conv3D(64, kernel_size=(3, 3, 3), padding='same'))
model2.add(Activation('relu'))
model2.add(MaxPooling3D(pool_size=(3, 3, 3), padding='same'))
model2.add(Dropout(0.25))

model2.add(Flatten())
model2.add(Dense(512, activation='relu'))
model2.add(Dropout(0.5))
model2.add(Dense(classes, activation='softmax'))

model2.compile(optimizer=keras.optimizers.Adam(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_13 (Conv3D)           (None, 30, 60, 60, 32)    2624      
_________________________________________________________________
activation_13 (Activation)   (None, 30, 60, 60, 32)    0         
_________________________________________________________________
conv3d_14 (Conv3D)           (None, 30, 60, 60, 32)    27680     
_________________________________________________________________
activation_14 (Activation)   (None, 30, 60, 60, 32)    0         
_________________________________________________________________
max_pooling3d_11 (MaxPooling (None, 10, 20, 20, 32)    0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 10, 20, 20, 32)    0         
_________________________________________________________________
conv3d_15 (Conv3D)           (None, 10, 20, 20, 64)    55360     
__________

In [44]:

train_generator = generator_ex(train_path, train_doc, batch_size)
val_generator = generator_ex(val_path, val_doc, batch_size)

In [45]:
model2.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  val ; batch size = 10
Source path =  Epoch 1/10
train ; batch size = 10
67/67 [==============================] - 99s 1s/step - loss: 1.6120 - categorical_accuracy: 0.2259 - val_loss: 1.5663 - val_categorical_accuracy: 0.1700

Epoch 00001: saving model to model_init_2020-03-2615_56_14.668735/model-00001-1.61206-0.22474-1.56632-0.17000.h5
Epoch 2/10
67/67 [==============================] - 34s 505ms/step - loss: 1.6058 - categorical_accuracy: 0.1990 - val_loss: 1.6128 - val_categorical_accuracy: 0.2100

Epoch 00002: saving model to model_init_2020-03-2615_56_14.668735/model-00002-1.60576-0.19900-1.61283-0.21000.h5
Epoch 3/10
67/67 [==============================] - 34s 505ms/step - loss: 1.6116 - categorical_accuracy: 0.2289 - val_loss: 1.6098 - val_categorical_accuracy: 0.2100

Epoch 00003: saving model to model_init_2020-03-2615_56_14.668735/model-00003-1.61162-0.22886-1.60979-0.21000.h5

Epoch 00003: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
Epo

## <font color='green'> Trial 2. Batch size = 20

In [46]:
train_generator = generator_ex(train_path, train_doc, 20)
val_generator = generator_ex(val_path, val_doc, 20)

In [47]:

model2.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  val ; batch size = 20
Source path =  train ; batch size = Epoch 1/10
20
67/67 [==============================] - 116s 2s/step - loss: 1.1097 - categorical_accuracy: 0.5401 - val_loss: 1.1047 - val_categorical_accuracy: 0.5600

Epoch 00001: saving model to model_init_2020-03-2615_56_14.668735/model-00001-1.10931-0.52625-1.10469-0.56000.h5
Epoch 2/10
67/67 [==============================] - 45s 678ms/step - loss: 1.0326 - categorical_accuracy: 0.5672 - val_loss: 1.1624 - val_categorical_accuracy: 0.4900

Epoch 00002: saving model to model_init_2020-03-2615_56_14.668735/model-00002-1.03264-0.56716-1.16239-0.49000.h5
Epoch 3/10
67/67 [==============================] - 46s 694ms/step - loss: 1.0659 - categorical_accuracy: 0.5274 - val_loss: 1.1534 - val_categorical_accuracy: 0.5100

Epoch 00003: saving model to model_init_2020-03-2615_56_14.668735/model-00003-1.06591-0.52736-1.15336-0.51000.h5

Epoch 00003: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.
Ep

## <font color='green'> Trial 3. Batchsize = 30

In [48]:
train_generator = generator_ex(train_path, train_doc, 30)
val_generator = generator_ex(val_path, val_doc, 30)

In [49]:
model2.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  val ; batch size = 30
Source path =  train ; batch size = 30
Epoch 1/10
67/67 [==============================] - 121s 2s/step - loss: 0.5862 - categorical_accuracy: 0.7576 - val_loss: 1.0238 - val_categorical_accuracy: 0.6000

Epoch 00001: saving model to model_init_2020-03-2615_56_14.668735/model-00001-0.57141-0.78239-1.02382-0.60000.h5
Epoch 2/10
67/67 [==============================] - 36s 535ms/step - loss: 0.5804 - categorical_accuracy: 0.7413 - val_loss: 1.0149 - val_categorical_accuracy: 0.5400

Epoch 00002: saving model to model_init_2020-03-2615_56_14.668735/model-00002-0.58036-0.74129-1.01490-0.54000.h5
Epoch 3/10
67/67 [==============================] - 37s 550ms/step - loss: 0.5388 - categorical_accuracy: 0.7811 - val_loss: 1.0149 - val_categorical_accuracy: 0.6200

Epoch 00003: saving model to model_init_2020-03-2615_56_14.668735/model-00003-0.53881-0.78109-1.01494-0.62000.h5
Epoch 4/10
67/67 [==============================] - 33s 492ms/step - loss: 0.4918 -

## <font color = 'green'> Trial 4. Batchsize = 40 and epoch =20

In [50]:
num_epochs = 20

In [51]:
train_generator = generator_ex(train_path, train_doc, 40)
val_generator = generator_ex(val_path, val_doc, 40)

In [52]:
model2.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  val ; batch size = 40
Source path =  Epoch 1/20train ; batch size = 40

67/67 [==============================] - 249s 4s/step - loss: 0.4798 - categorical_accuracy: 0.8128 - val_loss: 0.9900 - val_categorical_accuracy: 0.5875

Epoch 00001: saving model to model_init_2020-03-2615_56_14.668735/model-00001-0.47740-0.81565-0.99002-0.58750.h5
Epoch 2/20
67/67 [==============================] - 174s 3s/step - loss: 0.4655 - categorical_accuracy: 0.8227 - val_loss: 1.0475 - val_categorical_accuracy: 0.5550

Epoch 00002: saving model to model_init_2020-03-2615_56_14.668735/model-00002-0.46524-0.82318-1.04748-0.55500.h5
Epoch 3/20
67/67 [==============================] - 157s 2s/step - loss: 0.4698 - categorical_accuracy: 0.8209 - val_loss: 1.0050 - val_categorical_accuracy: 0.5950

Epoch 00003: saving model to model_init_2020-03-2615_56_14.668735/model-00003-0.46978-0.82090-1.00497-0.59500.h5

Epoch 00003: ReduceLROnPlateau reducing learning rate to 3.906250185536919e-06.
Epoch 

## <font color ='green'> Trial 5 . optimizer = Adadelta

In [53]:
num_epochs = 10

In [54]:
model2.compile(optimizer=keras.optimizers.Adadelta(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_13 (Conv3D)           (None, 30, 60, 60, 32)    2624      
_________________________________________________________________
activation_13 (Activation)   (None, 30, 60, 60, 32)    0         
_________________________________________________________________
conv3d_14 (Conv3D)           (None, 30, 60, 60, 32)    27680     
_________________________________________________________________
activation_14 (Activation)   (None, 30, 60, 60, 32)    0         
_________________________________________________________________
max_pooling3d_11 (MaxPooling (None, 10, 20, 20, 32)    0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 10, 20, 20, 32)    0         
_________________________________________________________________
conv3d_15 (Conv3D)           (None, 10, 20, 20, 64)    55360     
__________

In [55]:
train_generator = generator_ex(train_path, train_doc, 40)
val_generator = generator_ex(val_path, val_doc, 40)

In [56]:
model2.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  val ; batch size = 40
Source path =  train Epoch 1/10
; batch size = 40
67/67 [==============================] - 251s 4s/step - loss: 0.5409 - categorical_accuracy: 0.7940 - val_loss: 1.1986 - val_categorical_accuracy: 0.5792

Epoch 00001: saving model to model_init_2020-03-2615_56_14.668735/model-00001-0.57459-0.78551-1.19856-0.57917.h5
Epoch 2/10
67/67 [==============================] - 179s 3s/step - loss: 0.3950 - categorical_accuracy: 0.8540 - val_loss: 1.7383 - val_categorical_accuracy: 0.5900

Epoch 00002: saving model to model_init_2020-03-2615_56_14.668735/model-00002-0.39675-0.85408-1.73828-0.59000.h5
Epoch 3/10
67/67 [==============================] - 174s 3s/step - loss: 0.2942 - categorical_accuracy: 0.8859 - val_loss: 1.6281 - val_categorical_accuracy: 0.5550

Epoch 00003: saving model to model_init_2020-03-2615_56_14.668735/model-00003-0.29421-0.88586-1.62810-0.55500.h5

Epoch 00003: ReduceLROnPlateau reducing learning rate to 0.5.
Epoch 4/10
67/67 [======

## <font color ='green'> Trial 6. epoch = 20 

In [57]:
num_epochs = 20
model2.compile(optimizer=keras.optimizers.Adadelta(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_13 (Conv3D)           (None, 30, 60, 60, 32)    2624      
_________________________________________________________________
activation_13 (Activation)   (None, 30, 60, 60, 32)    0         
_________________________________________________________________
conv3d_14 (Conv3D)           (None, 30, 60, 60, 32)    27680     
_________________________________________________________________
activation_14 (Activation)   (None, 30, 60, 60, 32)    0         
_________________________________________________________________
max_pooling3d_11 (MaxPooling (None, 10, 20, 20, 32)    0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 10, 20, 20, 32)    0         
_________________________________________________________________
conv3d_15 (Conv3D)           (None, 10, 20, 20, 64)    55360     
__________

In [58]:
train_generator = generator_ex(train_path, train_doc, 40)
val_generator = generator_ex(val_path, val_doc, 40)

In [ ]:
model2.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  val Source path =  ; batch size = 40
Epoch 1/20
train ; batch size = 40
67/67 [==============================] - 245s 4s/step - loss: 0.1186 - categorical_accuracy: 0.9642 - val_loss: 2.0928 - val_categorical_accuracy: 0.6417

Epoch 00001: saving model to model_init_2020-03-2615_56_14.668735/model-00001-0.12469-0.96116-2.09280-0.64167.h5
Epoch 2/20
67/67 [==============================] - 158s 2s/step - loss: 0.1117 - categorical_accuracy: 0.9665 - val_loss: 2.0317 - val_categorical_accuracy: 0.6200

Epoch 00002: saving model to model_init_2020-03-2615_56_14.668735/model-00002-0.11157-0.96652-2.03175-0.62000.h5
Epoch 3/20
67/67 [==============================] - 173s 3s/step - loss: 0.0694 - categorical_accuracy: 0.9754 - val_loss: 2.1078 - val_categorical_accuracy: 0.5700

Epoch 00003: saving model to model_init_2020-03-2615_56_14.668735/model-00003-0.06938-0.97542-2.10783-0.57000.h5
Epoch 4/20
67/67 [==============================] - 159s 2s/step - loss: 0.0635 - categ

# Extra

In [9]:
num_epochs = 20

In [25]:
train_generator = generator_ex(train_path, train_doc, 64)
val_generator = generator_ex(val_path, val_doc, 64)

In [26]:
# Define model b
model2 = Sequential()
model2.add(Conv3D(32, kernel_size=(3, 3, 3), input_shape=(x,y,z,channel), padding='same'))
model2.add(Activation('relu'))
model2.add(Conv3D(32, kernel_size=(3, 3, 3), padding='same'))
model2.add(Activation('relu'))
model2.add(MaxPooling3D(pool_size=(3, 3, 3), padding='same'))
model2.add(BatchNormalization())

model2.add(Conv3D(64, kernel_size=(3, 3, 3), padding='same'))
model2.add(Activation('relu'))
model2.add(Conv3D(64, kernel_size=(3, 3, 3), padding='same'))
model2.add(Activation('relu'))
model2.add(MaxPooling3D(pool_size=(3, 3, 3), padding='same'))
model2.add(BatchNormalization())

model2.add(Conv3D(128, kernel_size=(3, 3, 3), padding='same'))
model2.add(Activation('relu'))
model2.add(Conv3D(128, kernel_size=(3, 3, 3), padding='same'))
model2.add(Activation('relu'))
model2.add(MaxPooling3D(pool_size=(3, 3, 3), padding='same'))
model2.add(BatchNormalization())

model2.add(Flatten())
model2.add(Dense(512, activation='relu'))
model2.add(Dropout(0.5))
model2.add(Dense(64, activation='relu'))
model2.add(Dense(classes, activation='softmax'))

model2.compile(optimizer=keras.optimizers.Adam(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_19 (Conv3D)           (None, 30, 60, 60, 32)    2624      
_________________________________________________________________
activation_19 (Activation)   (None, 30, 60, 60, 32)    0         
_________________________________________________________________
conv3d_20 (Conv3D)           (None, 30, 60, 60, 32)    27680     
_________________________________________________________________
activation_20 (Activation)   (None, 30, 60, 60, 32)    0         
_________________________________________________________________
max_pooling3d_12 (MaxPooling (None, 10, 20, 20, 32)    0         
_________________________________________________________________
batch_normalization_5 (Batch (None, 10, 20, 20, 32)    128       
_________________________________________________________________
conv3d_21 (Conv3D)           (None, 10, 20, 20, 64)    55360     
__________

In [27]:
model2.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=20, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  val ; batch size =Source path =  train ; batch size = Epoch 1/20 64
64

Batch:  2 Index: 64
67/67 [==============================] - 261s 4s/step - loss: 1.3593 - categorical_accuracy: 0.4530 - val_loss: 3.0489 - val_categorical_accuracy: 0.3101

Epoch 00001: saving model to model_init_2020-03-2607_25_29.400376/model-00001-1.42852-0.42469-3.04888-0.31013.h5
Epoch 2/20
67/67 [==============================] - 146s 2s/step - loss: 1.1367 - categorical_accuracy: 0.5308 - val_loss: 2.1508 - val_categorical_accuracy: 0.3100

Epoch 00002: saving model to model_init_2020-03-2607_25_29.400376/model-00002-1.13795-0.52905-2.15082-0.31000.h5
Epoch 3/20
67/67 [==============================] - 146s 2s/step - loss: 0.9393 - categorical_accuracy: 0.6093 - val_loss: 0.9489 - val_categorical_accuracy: 0.6000

Epoch 00003: saving model to model_init_2020-03-2607_25_29.400376/model-00003-0.93928-0.60931-0.94894-0.60000.h5
Epoch 4/20
67/67 [==============================] - 143s 2s/step - 